In [3]:
import requests

url = r'https://gatherer.wizards.com/Pages/Search/Default.aspx?page=0&name=+[]'

# gibt html code der gewünschten url zurück
def get_url_content(url):
    return requests.get(url).text

In [25]:
import requests
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import numpy as np
from bs4 import BeautifulSoup
import math

disable_warnings(InsecureRequestWarning)

In [ ]:
page = requests.get(url, verify=False)

print(page.content)

In [16]:
prev = ''
i = 0
while(i < 2):
    data = requests.get(url, verify=False)
    if data != prev:
        prev = data
        i += 1
    else:
        exit()

In [20]:
data = []
i = 0
while(i < 1):
    data.append(requests.get("https://gatherer.wizards.com/Pages/Search/Default.aspx?page="+str(i)+"&name=+[]", verify=False))
    i += 1

In [21]:
async def hallo():
    pass



In [26]:
import asyncio
from time import perf_counter
import aiohttp

In [27]:
def __checkForEndOfPages(html, page):
    """checks if the underlined pagenumber in the @html string is equivilant to the number given as @page

    Args:
        html (str): html-string containing one page
        page (int): page number

    Returns:
        bool: if the page numbers are the same -> False (not end of Pages). If page numbers are not the same -> True
    """
    return int(BeautifulSoup(html, 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0]) != page

def __deleteRedundantPages(res, div):
    """delete all pages from the end, that are redundant

    Args:
        res (list of string): list that contains all pages
        div (int): amount of pages per iteration

    Returns:
        list of string: list that contains all pages (now without redundant pages at the end)
    """
    if div == 1:
        res = res[:-1]
    else:
        buf = res[-div:]
        res = res[:-div]
        
        parse_buffer1 = int(BeautifulSoup(buf[-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0])
        for i in reversed(range(1, div)):
            parse_buffer2 = int(BeautifulSoup(buf[i-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0])
            if parse_buffer1 == parse_buffer2:
                buf.pop()
                if i == 1 and len(res) > 0:
                    if int(BeautifulSoup(buf[i-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0]) == int(BeautifulSoup(res[-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0]):
                        buf.pop()
            else:
                break
            parse_buffer1 = parse_buffer2
        res += buf
    return res

async def fetch(s, url):
    async with s.get(f'https://gatherer.wizards.com/Pages/Search/Default.aspx?sort=cn+&page={url}&name=%20[]') as r:
        if r.status != 200:
            r.raise_for_status()
        return await r.text()

async def fetch_all(s, pageNum, offset, div):
    tasks = []
    res = []
    end = math.ceil(pageNum/div)
    if pageNum != 0:
        for i in range(int(offset/div), end + int(offset/div)):
            for i2 in range(i*div, (i + 1)*div):
                task = asyncio.create_task(fetch(s, i2))
                tasks.append(task)
            res = await asyncio.gather(*tasks)
            print(i)
            if __checkForEndOfPages(res[-1], (i + 1)*div):
                res = __deleteRedundantPages(res, div)
                break  
        if pageNum < len(res):
            res = res[:pageNum]
    else:
        i = int(offset/div)
        while 1:
            for i2 in range(i*div, (i + 1)*div):
                task = asyncio.create_task(fetch(s, i2))
                tasks.append(task)
            res = await asyncio.gather(*tasks)
            print(i)
            if __checkForEndOfPages(res[-1], (i + 1)*div):
                res = __deleteRedundantPages(res, div)
                break
            i += 1
    return res

async def main(pageNum = 0):
    div = 10        # integer
    offset = 249    # integer - sollte Vielfaches von div sein - bei div = 10 -> offset = 10 oder = 20 oder ... oder = 250
                    # wird als offset = 241 gewählt, so wird intern daraus 240 gemacht, genauso bei offset = 242 oder ... oder 249 ... usw.
    if div < 1:
        print("Div muss >= 1")
    else:
        async with aiohttp.ClientSession() as session:
            htmls = await fetch_all(session, pageNum, offset, div)
            #print(htmls)
            return htmls

start = perf_counter()
#asyncio.run(main())
htmls = await main(0)
stop = perf_counter()
print("time taken:", stop - start)

24
25
time taken: 26.02024579999852


In [34]:
__checkForEndOfPages(htmls[0], 1)

True

In [28]:
len(htmls)

17

In [12]:
len(htmls[-10:])

10

In [7]:
import re

In [6]:
import re
results = BeautifulSoup(htmls[0], 'html.parser')

In [129]:
print(re.search("multiverseid=(.*)&", url).group(1))

2344353


In [ ]:
results

In [30]:
def getImageAndID(card):
    data = card.find('td', class_='leftCol').find('a').find('img')['src'][5:]
    return "https://gatherer.wizards.com" + data, re.search("multiverseid=(.*)&", data).group(1)

In [31]:
def getName(card):
    return card.find('td', class_='middleCol').find('div', class_='cardInfo').find('span', class_='cardTitle').find('a').contents[0]

In [32]:
def getType(card):
    return re.sub(' +', ' ', re.sub(r'\r\n', "", str(card.find('td', class_='middleCol').find('div', class_='cardInfo').find('span', class_='typeLine').contents[0])))

In [82]:
cards = results.find_all('tr', class_=['cardItem evenItem', 'cardItem oddItem'])

In [ ]:
cards

In [ ]:
cards[0]

In [79]:
len(cards)

100

In [154]:
re.sub(' +', ' ', re.sub(r'\r\n', "", getType(cards[1])))

' Creature — Human Cleric (1/3)'

In [ ]:
import re

for html in htmls:
    results = BeautifulSoup(html, 'html.parser')
    cards = results.find_all('tr', class_=['cardItem evenItem', 'cardItem oddItem'])
    #print(i)
    for card in cards:
        print(getName(card))
        print(getImageAndID(card))
        print(getType(card))

In [43]:
class csvCreator():
    
    def __init__(this):
        this.__csvCols = []
        this.__csvConstruct = []
        this.__csv = ''
    
    def defineCol(this, *params):
        this.__csvCols = []
        for param in params:
            this.__csvCols.append(param)
    
    def addCard(this, *params):
        this.__csvConstruct.append([])
        for param in params:
            this.__csvConstruct[-1].append(param)
    
    def getCsv(this):
        this.__csv = this.__printRow(this.__csvCols)
        for row in this.__csvConstruct:
            this.__csv += this.__printRow(row)
        return this.__csv
    
    def __printRow(this, row):
        buf = ""
        for valorem in row:
            buf += str(valorem) + ","
        return buf[:-1] + "\n"
        

In [42]:
csv = csvCreator()
csv.defineCol("id", "name")

In [22]:
csv.addCard(1, "Adrian")
csv.addCard(2, "Adrian2")

In [23]:
print(csv.getCsv())

id,name
1,Adrian
2,Adrian2



In [ ]:
# parse html-input as json

for html in htmls:
    results = BeautifulSoup(html, 'html.parser')
    cards = results.find_all('tr', class_=['cardItem evenItem', 'cardItem oddItem'])
    #print(i)
    for card in cards:
        print(getName(card))
        print(getImageAndID(card))
        print(getType(card))

In [45]:
# combined crawler-data with csvCreator

import re

csv = csvCreator()
csv.defineCol("ID", "Image-ID", "Name", "Type")

for html in htmls:
    results = BeautifulSoup(html, 'html.parser')
    cards = results.find_all('tr', class_=['cardItem evenItem', 'cardItem oddItem'])
    #print(i)
    for card in cards:
        img, id = getImageAndID(card)
        csv.addCard(id, img, getName(card), getType(card))
    
f = open("Lukas.csv", "w")
f.write(csv.getCsv())
f.close()